https://note.com/npaka/n/na5b8e6f749ce

x https://note.com/masuidrive/n/n0e2a11fc5bfa

In [1]:
# ! wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
# ! tar xf ldcc-20140209.tar.gz
# ! rm ldcc-20140209.tar.gz

In [3]:
# 基本パラメータ
model_name = "cyberagent/open-calm-medium"
peft_name  = "lora-calm-medium"
output_dir = "lora-calm-medium-results"

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForCausalLM.from_pretrained(model_name)

/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(tokenizer.special_tokens_map)
print("bos_token :", tokenizer.eos_token, ",", tokenizer.bos_token_id)
print("eos_token :", tokenizer.bos_token, ",", tokenizer.eos_token_id)
print("unk_token :", tokenizer.unk_token, ",", tokenizer.unk_token_id)
print("pad_token :", tokenizer.pad_token, ",", tokenizer.pad_token_id)

{'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|padding|>'}
bos_token : <|endoftext|> , 0
eos_token : <|endoftext|> , 0
unk_token : <|endoftext|> , 0
pad_token : <|padding|> , 1


In [4]:
CUTOFF_LEN = 512  # 最大長

# トークナイズ関数の定義
def tokenize(prompt, tokenizer):
    result = tokenizer(
        prompt+"<|endoftext|>",  # EOSの付加
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=True,
    )
    return {
        "input_ids": result["input_ids"],
        "attention_mask": result["attention_mask"],
    }

In [22]:
# トークナイズの動作確認
tokenize("hi there", tokenizer)

{'input_ids': [40254, 36767, 0], 'attention_mask': [1, 1, 1]}

## データセットの作成

In [30]:
import os
import glob

dirname2label = {
    'dokujo-tsushin': "独女通信",
    'it-life-hack': "ITライフハック",
    'kaden-channel': "家電チャンネル",
    'livedoor-homme': "livedoor HOMME",
    'movie-enter': "MOVIE ENTER",
    'peachy': "Peachy",
    'smax': "エスマックス",
    'sports-watch': "Sports Watch",
    'topic-news': "トピックニュース",
}

data_all = []
for dirname, label in dirname2label.items():
    os.path.join('text', dirname)
    filepaths = glob.glob(os.path.join('text', dirname, '*.txt'))
    for filepath in filepaths:
        text = open(filepath, 'r', encoding='utf-8').read()
        # 内容のある最初の３文だけ取り出す
        lines = []
        for line in text.split('\n')[2:]:
            if len(line) > 0:
                lines.append(line)
                if len(lines) >= 3:
                    break
        text = '\n'.join(lines) + '\n'
        data_all.append({'text': text, 'label': label})

In [6]:
def generate_prompt(data_point):
    return f"""{data_point['text']}

### Label:
{data_point['label']}"""

In [7]:
print(generate_prompt(data_all[0]))

森ガールよりもリア充？ 「雲ガール」の実態に迫る
秋深き　隣は何を　する人ぞ　——。これまで「森ガール」にはじまり「山ガール」、「釣りガール」とさまざまな流行を生みだしてきた“○○ガールズ”ブーム。2011年も本格的に秋の様相を呈してきた今、新たなガールズブームが産声を上げた。その名も「雲ガール」。これはPCやスマートフォンでインターネットを活用し、趣味や生活を楽しんでいる女子たちのこと。ちなみに「雲」は、今もっともホットなインターネットサービス「クラウド」を指すのである。
今回、この雲ガールたちの座談会があるとの情報をキャッチ！　潜入し、雲ガールの実態をチェックしてきたので、ここに報告しよう。


### Label:
独女通信


In [16]:
from datasets import Dataset
import pandas as pd
df_data_all = pd.DataFrame(data_all)
dataset_all = Dataset.from_pandas(df_data_all)

In [17]:
VAL_SET_SIZE = 200

# 学習データと検証データの準備
train_val = dataset_all.train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]
train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))
val_data = val_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))


In [18]:
from transformers import AutoModelForCausalLM

# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/fujie/.conda/envs/py39 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


In [19]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# LoRAのパラメータ
lora_config = LoraConfig(
    r= 8, 
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# モデルの前処理
model = prepare_model_for_int8_training(model)

# LoRAモデルの準備
model = get_peft_model(model, lora_config)

# 学習可能パラメータの確認
model.print_trainable_parameters()

trainable params: 786432 || all params: 409790464 || trainable%: 0.19191076149590441


In [20]:
import transformers
eval_steps = 200
save_steps = 200
logging_steps = 20

# トレーナーの準備
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        num_train_epochs=3,
        learning_rate=3e-4,
        logging_steps=logging_steps,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=eval_steps,
        save_steps=save_steps,
        output_dir=output_dir,
        report_to="none",
        save_total_limit=3,
        push_to_hub=False,
        auto_find_batch_size=True
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [21]:
# 学習の実行
model.config.use_cache = False
trainer.train() 
model.config.use_cache = True

# LoRAモデルの保存
trainer.model.save_pretrained(peft_name)

/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [26]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

# トークンナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(model_name)

# LoRAモデルの準備
model = PeftModel.from_pretrained(
    model, 
    peft_name, 
    device_map="auto"
)

# 評価モード
model.eval()

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(52096, 1024)
        (layers): ModuleList(
          (0-23): 24 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear8bitLt(
                in_features=1024, out_features=3072, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                

In [28]:
def generate_prompt_q(data_point):
    return f"""{data_point['text']}

### Label:
"""

In [76]:
# テキスト生成関数の定義
def generate(instruction,input=None,maxTokens=512):
    # 推論
    prompt = generate_prompt_q({'text':instruction})
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model.generate(
        input_ids=input_ids, 
        max_new_tokens=maxTokens, 
        do_sample=True,
        temperature=0.7, 
        top_p=0.75, 
        top_k=40,         
        no_repeat_ngram_size=2,
    )
    outputs = outputs[0].tolist()

    # EOSトークンにヒットしたらデコード完了
    if tokenizer.eos_token_id in outputs:
        eos_index = outputs.index(tokenizer.eos_token_id)
        decoded = tokenizer.decode(outputs[:eos_index])

        # ラベル内容のみ抽出
        sentinel = "### Label:"
        sentinelLoc = decoded.find(sentinel)
        if sentinelLoc >= 0:
            response = decoded[sentinelLoc+len(sentinel):]
            # print(response)
        else:
            print('Warning: Expected prompt template to be emitted.  Ignoring output.')
            response = None
    else:
        print('Warning: no <eos> detected ignoring output')
        response = None
        decoded = None

    return response, prompt, decoded

In [88]:
d = data_all[6000]

response, prompt, decoded = generate(d['text'])
print("---- PROMPT ----")
print(prompt)
print("---- RESPONSE ----")
print(response)
print("---- TARGET ----")
print(d['label'])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


---- PROMPT ----
スペインからも苦情が出た日本のテレビ中継
セレモニー中のメッシをおかしな質問で呼び止めて大きな批判を浴びるなど、クラブワールドカップ2011の放送はネット上であまり評判がよくなかった。しかし、実はバルセロナの本国であるスペインでもこの放送にはブーイングが起きていたようだ。
スペインの事情に詳しいサッカージャーナリストの小澤一郎氏によると、スペインにおける評判は以下のようなものだったという。


### Label:

---- RESPONSE ----

Sports
---- TARGET ----
Sports Watch
